In [1]:
# Download tesseract-ocr file for system installation:
#        https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-setup-3.05.00dev.exe
#        - download the above exe and run it
#        - while installing the setup will ask to download additional languages
#          (if you want to process images in different languages also like French, etc.) - by default english is there

!pip install pillow
!pip install pytesseract
!pip install opencv-python
!pip install pdf2image

In [7]:
# import the necessary packages
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import glob

L_jpg=glob.glob("C://Users//Acer//Desktop//ocr//downloaded_attachments//*.jpg")
L_png=glob.glob("C://Users//Acer//Desktop//ocr//downloaded_attachments//*.png")
L_jpg.extend(L_png)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'

# --------------- load --------------
c=2
while c:
    f=open('output.txt','w')
    for img in L_jpg:
        image = cv2.imread(img)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # ---------------------------------------

        # ---------------- preprocess ------------------
        # check to see if we should apply thresholding to preprocess the
        # image
        pre=["thresh","blur"]
        sel_pre=pre[0]
        if sel_pre == "thresh":
            gray = cv2.threshold(gray, 0, 255,
                cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

        # make a check to see if median blurring should be done to remove noise
        elif sel_pre == "blur":
            gray = cv2.medianBlur(gray, 3)

        # write the grayscale image to disk as a temporary file so we can
        # apply OCR to it
        filename = "{}.png".format(os.getpid())
        cv2.imwrite(filename, gray)
        # ---------------------------------------
        print('Temporary File:',filename)
        # ---------------- OCR ------------------
        # load the image as a PIL/Pillow image, apply OCR, and then delete
        # the temporary file
        text = pytesseract.image_to_string(Image.open(filename))
        os.remove(filename)

        # ------------ json -------------------

        texts=text.replace(' ','').split('\n')
        tt=text.lower()
        if 'order' in tt:
            doctype="PO"
            f.write("#PurchaseOrder\n")
        #     print("order")
        elif 'invoice' in tt:
            doctype="INVOICE"
            f.write("#Invoice\n")
        #     print("invoice")

        data=[]
        for i in texts:
            if i!='':
                data.append(i)
        json="{\n"
        json+="\t\"doctype\":\""+doctype+"\",\n"
        for i in data:
            x=i.split(':')
            if len(x)>1:
                json+="\t\""+x[0]+"\":\""+' '.join(x[1:])+"\",\n"
        json+="}"
       # print(json)
        f.write(json+'\r\n')
    c-=1


Temporary File: 7528.png
Temporary File: 7528.png
Temporary File: 7528.png
Temporary File: 7528.png
Temporary File: 7528.png
Temporary File: 7528.png


In [2]:
import imaplib,email
user=input('Enter gmail id: ')
password=input('Enter password: ')
imap_url = 'imap.gmail.com'

attachment_dir = 'downloaded_attachments'
import os
if not os.path.exists(attachment_dir):
    os.makedirs(attachment_dir)

mail = imaplib.IMAP4_SSL(imap_url)
mail.login(user, password)
mail.list()
# Out: list of "folders" aka labels in gmail.
mail.select("inbox") # connect to inbox.

# -------------------------------------------- #
# allows you to download attachments
def get_attachments(msg):
    for part in msg.walk():
        if part.get_content_maintype()=='multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        fileName = part.get_filename()
        if bool(fileName):
            filePath = os.path.join(attachment_dir, fileName)
            with open(filePath,'wb') as f:
                f.write(part.get_payload(decode=True))


result, data = mail.uid('search', None, "ALL") # search and return uids instead
last_50_ids = data[0].split()[-10:-1]#-50:0][::-1]
latest_id = data[0].split()[-1]
last_50_ids.append(latest_id)
# print(last_50_ids)
last_50_ids=last_50_ids[::-1]
c=1
for i in last_50_ids:
    print ('Fetching mail:',c)
    result, data = mail.uid('fetch', i, '(RFC822)')
    raw_email = data[0][1]
    # print(raw_email)
    # --------------------- formatting ------------ #
    email_message = email.message_from_bytes(raw_email)
    print ('\tTo',email_message['To'])
    print ('\tFrom',email.utils.parseaddr(email_message['From']) )# for parsing "Yuji Tomita" <yuji@grovemade.com>
    # print (email_message.items()) # print all headers
     # --------------------------------------------------- #
    get_attachments(email_message)
    print('\tDowloaded attachments for this mail !')
    c+=1


Enter gmail id: vaishwiksanghavi@gmail.com
Enter password: Kingfisher20000@
Fetching mail: 1


AttributeError: 'bytes' object has no attribute 'message_from_strings'